# Introduction

In this notebook, I will attempt to predict whether or not a terrorist attack will occur in a given country conditional on certain socioeconomic variables, as well as other variables which are thought to be relevant to the incidence of terrorism. To do this, I will train some classifiers on data I collected for my undergraduate thesis in international relations.

This notebook covers some, but not all, of the data analysis work I performed in my thesis. For instance, I did not include any of the fixed effects logistic regression models that were in my thesis, because to the best of my knowledge, this type of model has not yet been implemented in Python. 

It's also important to note that I performed the data analysis in my thesis using R and Stata, but the analysis in this notebook is deliberately written in pure Python. The reason for this is because I suspect that any production code at Open Data Nation has to be written entirely in Python. As such, the performance of the classifiers in this notebook may be slightly different from what I discussed in my technical interview due to differences in how R and Python implement machine learning models. Nevertheless, I will make a reasonable effort to train models that achieve excellent results on the relevant classification metrics.

Before I delve into the analysis, a disclaimer. The models here are not as "fine-tuned" as they would be if I was working in a professional setting because this work was done in a day. I would definitely do more parameter tuning to create better algorithms or try some more computationally expensive classifiers if I had more time to work on this notebook. With that being said, the models trained in this notebook should be pretty good for a first attempt. I think that this notebook is, all in all, a good representation of my Python data science skills.


# A description of the data

*This section is a condensed excerpt from my thesis. The cited papers/books are available on request.*

Before I describe the variables in detail, it is helpful to understand what my hypotheses, and by extension, my models, are trying to measure. I have two hypotheses about the relationship between ethnic tensions, income inequality, and terrorism; as I mentioned earlier, one relates ethnic tension and income inequality to the “risk” of terrorism, while the other relates those factors to the number of attacks that are carried out. The former refers to the likelihood that a terrorist attack will occur conditional on a number of factors, while the latter is self-explanatory; each one is estimated in a separate model. 

This brings me to the description of my dependent variables, both of which come from START’s Global Terrorism Database (henceforth referred to as GTD). A terrorist attack in the GTD is defined as “the threatened or actual use of illegal force and violence by a non‐state actor to attain a political, economic, religious, or social goal through fear, coercion, or intimidation” (GTD Codebook: Inclusion Criteria and Variables, 8). I constructed the binary dependent variable by first cleaning the data set so that it only included cases in which there is no doubt that the incident was a terrorist attack. Furthermore, because I focus on the post-Cold War era in my analyses, I only include terrorist attacks that occur after 1991, as the Soviet Union officially collapsed on December 25 of that year. There are no cases for 1993, as the data for that year were lost. I code the variable as “0” if there were no terrorist attacks for a given country-year, and as “1” otherwise. In its second iteration, I code the variable as “0” if there were less than five terrorist attacks in a given country-year, and as “1” otherwise. 

In the hypothesis section, I posit that the interaction between ethnic tension and income inequality is the main contributing factor to the risk of a terrorist attack occurring. Because these variables are at the heart of my hypothesis, I was careful to find sources that most accurately captured the two phenomena. Most empirical studies that include income inequality operationalize it as the Gini coefficient, and I basically do the same thing. Like the Gini coefficient in its usual form, my measure of income inequality is a continuous variable that ranges from 0 to 1, with the former value expressing perfect income equality, and the latter value expressing perfect income inequality. In order to better capture the distribution of income in a given country-year, I use data from Frederick Solt’s (2014) Standardized World Income Inequality Database (henceforth referred to as the SWIID). The SWIID has two advantages over other Gini coefficient data: first, it strikes the greatest possible balance between coverage and comparability, which has traditionally been a problem when comparing income inequality over space and time (Solt 2014, 2). Secondly, Solt’s data set includes a post-tax, post-transfer Gini coefficient for a large number of country-years. This is a major improvement over the usual way that income inequality is calculated, because it ensures that the Gini coefficient reflects the effects of redistributive policies on the distribution of income, which in turn comes closer to capturing the true distribution of income.  

I operationalize ethnic tension in two different ways because there are advantages to using each operationalization. The first operationalization is a direct measure of ethnic tensions, while the other, ethnic fractionalization (or ELF), is a proxy for ethnic tension. I would have preferred to stick to the former, but it only covers 88 countries total, whereas the ELF data covers all sovereign states with a population of 500,000 in 1990, which represents a majority of the world’s countries. I concede that ELF is not a perfect correlate of ethnic tension ($r \approx 0.56$), but utilizing it allows me to see if using the small sample of countries in the ethnic tension data severely biases the parameter estimates of my models. If the sample is representative of the population, then at the very least the direction of those parameter estimates should be the same as the ELF models.
The direct measure of ethnic tension comes from the PRS Group, a political risk consulting firm. The PRS Group’s data, which were generated by an unknown, proprietary process, provide scores of ethnic tension for a country-month on a scale of 6 to 1, which is further divided into increments of 0.5. The codebook for the data states that “Lower ratings are given to countries where racial and nationality tensions are high because opposing groups are intolerant and unwilling to compromise. Higher ratings are given to countries where tensions are minimal, even though such differences may still exist” (International Country Risk Guide Methodology, 6). For the sake of easier interpretation, I remapped the values such that a lower score corresponds to lower ethnic tensions. Finally, I aggregated the data into country-years by averaging the ethnic tension scores of each country-month. 

The indirect measure of ethnic tension is the ELF index as used by Alesina et al. (2003, 159). The ELF represents the probability that “two randomly selected individuals from a population belong to different groups” (Alesina et al. 2003, 158-159). Because I’m using panel data, the equation for calculating ELF is modified slightly:

$$ ELF_{jk} = 1 - \sum_{i=1}^{n} s^2_{ijk} $$

where $s_{ijk}$ is the share of group $i$ in country $j$ during year $k$. I wasn’t able to find panel data on the exact ethnic composition of countries, so I constructed my ELF variable from the 2014 Ethnic Power Relations (EPR) data set. The EPR data set records the sizes of politically relevant ethnic groups relative to the respective population of their country. An ethnic group is politically relevant if “...either at least one significant political actor claims to represent the interests of that group in the national political arena or if group members are systematically and intentionally discriminated against in the domain of public politics.” A significant political actor is defined as “a political organization (not necessarily a party) that is active in the national political arena”, while discrimination is “political exclusion directly targeted at an ethnic community” (Vogt, 3).
	
Scholars occasionally cite abject poverty as a reason for why people resort to terrorism, and as I alluded to in the introductory section, it is still one of the explanations that holds currency among senior U.S. policymakers. Although it seems apparent that poverty per se doesn’t cause terrorism, the fact that it still holds currency as an explanation for the roots of terrorism means that it’s important to control for in my regressions. Consequently, I use the World Bank’s data set on gross domestic product per capita (GDP per capita) to do so, which is measured in millions of constant 2005 US Dollars. I chose GDP per capita over pure GDP because the former scales the latter with respect to population; using purely GDP would produce skewed estimates of wealth. Using GDP to compare economic development in China and the U.S. illustrates this point perfectly. Finally, to better fit the data to a line, I take the natural log of each observation.

GDP is not the only indicator of economic development that scholars have proposed as a determinant of terrorism. Besides wealth and income inequality, foreign aid, globalization, and access to higher education have also been put forth as predictors of terrorism, and so I control for these factors. To maximize comparability between my analyses and those of Li and Schaub and Azam and Thelen, I use the same World Bank data sets on globalization and education.

Like Li and Schaub, I operationalize globalization as trade openness. Trade openness is measured as “the sum of exports and imports of goods and services measured as a share of gross domestic product” (“Trade (% of GDP)”). Like Azam and Thelen, I operationalize access to higher education as the gross secondary school enrollment ratio. The World Bank defines this ratio as “the total enrollment in secondary education, regardless of age, expressed as a percentage of the population of official secondary education age.” The ratio can exceed 100 percent, as it includes over-aged and under-aged students in its calculation (“School Enrollment, Secondary (% Gross)”). Using the net secondary enrollment ratio, which measures the percent of secondary-school age children who are enrolled in secondary school, may be a better indicator of how well a country’s children are being educated. However, comparability seems more important here, so I use the gross enrollment ratio instead. Finally, because Azam and Thelen’s foreign aid data do not cover the same time period that I do in my thesis, and because no readymade data set already exists for it, I constructed my own data set on foreign aid per capita by dividing the World Bank’s data on net official development assistance (ODA) and official aid, which are in constant 2011 US Dollars, by the midyear population of each country, which also comes from the World Bank. This calculation is very similar to the one that the World Bank uses to calculate net ODA per capita, so I am confident that I am measuring foreign aid flows properly.

A number of political factors have been put forth to explain why people engage in terrorism. Here, I focus on two of the most prominent political explanations that the literature discusses: regime stability and political and economic liberties. The data on regime stability, which comes from Polity IV’s DURABLE variable, measures “The number of years since the most recent regime change...or the end of [sic] transition period defined by the lack of stable political institutions” (Marshall et al., 17). I use two different data sources for political liberties in my models. The first source is Freedom House, whose political rights and civil liberties score I average to get a composite score of political liberties. Political rights and civil liberties are scored on a scale of 1 to 7, with 1 representing the smallest degree of freedom and 7 representing the greatest degree of freedom. The second source is Polity IV’s POLITY2 score. The POLITY2 score is identical to the POLITY score, except for the fact that it has been adjusted for time-series analyses. Thus, it is still measured on a scale of -10 to 10, with higher scores representing more democratic polities. Also, periods of interruption are coded as missing values, periods of anarchy are coded as 0, and “cases of ‘transition’ are prorated across the span of the transition.” Finally, the codebook for Polity IV notes that “Ongoing (-88) transitions in the most recent year (2006) are converted to [missing] values. Transitions (-88) following a year of independence, interruption (-66), or interregnum (-77) are prorated from the value ‘0’” (Marshall et al., 17).

Finally, scholars propose a number of variables, most of which are geographic, that have to do with a country’s capability to apprehend terrorists. These don’t explain why individuals resort to terrorism, but they are a standard control in the terrorism literature, so I control for them as well. Abadie mentions that mountainous areas and tropical jungles make it more difficult for authorities to catch terrorists, so I include data on the average elevation of a country and the percent of its area that’s comprised of geographical tropics, both of which come from Gallup’s Physical Geography data. Eyerman (1998) argues that policing costs increase with a country’s population, which makes it harder for authorities to apprehend terrorists. I operationalize this by taking the natural logarithm of the World Bank’s data on midyear population.



# Building a Classifier

Now I will train some models in an attempt to predict whether a terrorist attack will occur in a given country-year, e.g., if a terrorist attack happens in Algeria in 1997 or France in 2001. Terrorist attacks are (in most countries) rare events, so I will focus on predicting whether any attacks happen rather than predicting the number of attacks. It turns out that there isn't much variation in the number of attacks, so simply predicting if an attack will happen does make sense in this context. 

I should also note that the analysis below focuses on prediction rather than explanation, so I do not claim that any of the variables in these models are *causes* of terrorism. However, they could indirectly influence whether an attack happens, so it makes sense to include them in a predictive model. Furthermore, associations between variables provide a useful starting point for investigating causal relationships, so strong predictive results may provide direction in the hunt for causal mechanisms. That is where the value of this analysis lies.

My goals in building a classifier are threefold. First and foremost, I am interested in building a classifier that scores well on suitable metrics, like recall or area under the ROC (AUROC). Secondary to this goal is building a model that is highly interpretable so that clients, coworkers, and other interested parties can easily scrutinize it if they wish. A final goal is to build a predictive model that still works well even when its assumptions can't be verified. The best way to balance these three goals in my opinion is to build predictive models using decision trees and the k-nearest neighbor classifier. In the code below, I will train and tune both models to ensure that they work on out-of-sample data.

In [62]:
# Python 3.5.6. All packages updated to the latest version.
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
import os

os.chdir('/Users/Lee/Downloads')
data = pd.read_csv('thesis_dataset.csv')

In [8]:
data.head(5)

,Unnamed: 0,country,year,terroristattack,nattacks,gini_net,ethnictension,gdppc,polliberties,tradeopenness,lnpopulation,durable,elf,education,aid_x,elev,lcr100km,tropicar
0,1,Algeria,1992,1,145,33.553867,3.0,1743.346069,6.5,49.189084,17.130974,0,0.4032,56.96680,13.526710,572.802795,0.043235,0.166656
1,2,Algeria,1994,1,125,32.123112,4.0,1479.600215,7.0,48.584438,17.174244,2,0.4032,57.85013,10.212937,572.802795,0.043235,0.166656
2,3,Algeria,1995,1,134,31.407736,4.0,1424.642484,6.0,55.191005,17.193626,0,0.4032,57.45376,10.025426,572.802795,0.043235,0.166656
3,4,Algeria,1996,1,98,31.768595,4.0,1572.831937,6.0,53.705148,17.211535,1,0.4032,57.26973,10.220736,572.802795,0.043235,0.166656
4,5,Algeria,1997,1,201,32.129452,4.0,1587.646138,6.0,52.243912,17.228158,2,0.4032,57.27395,8.166294,572.802795,0.043235,0.166656


In [63]:
data=data.drop(['Unnamed: 0'], axis=1)

In [20]:
data.dtypes

country             object
year                 int64
terroristattack      int64
nattacks             int64
gini_net           float64
ethnictension      float64
gdppc              float64
polliberties       float64
tradeopenness      float64
lnpopulation       float64
durable              int64
elf                float64
education          float64
aid_x              float64
elev               float64
lcr100km           float64
tropicar           float64
dtype: object

In [64]:
data=data.drop(['country', 'year'], axis=1)


In [58]:
data.dtypes

terroristattack      int64
nattacks             int64
gini_net           float64
ethnictension      float64
gdppc              float64
polliberties       float64
tradeopenness      float64
lnpopulation       float64
durable              int64
elf                float64
education          float64
aid_x              float64
elev               float64
lcr100km           float64
tropicar           float64
dtype: object

Now that the data is loaded and the variables are changed to the right data types, I will start the model building process.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

dtree = tree.DecisionTreeClassifier()
k_neighbors = KNeighborsClassifier()

data = data.reset_index()
data.dropna()

y, X = data['terroristattack'], data.loc[:, data.columns != 'terroristattack']
X=np.nan_to_num(X)

k_neighbors.fit(X,y)
dtree.fit(X,y)

In [34]:
dtree.get_params()

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [36]:
k_neighbors.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': 1,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

As you can see, both models offer plenty of parameters to tune. For the sake of time, I will only tune 'n_neighbors' on my k-nearest neighbors algorithm (this determines how many neighbors each point takes into consideration when classifying) and 'min_samples_split' on my decision tree, which determines the minimum number of samples required to split an internal node.

In my experience, a grid search is a highly inefficient way to perform parameter tuning. A randomized search often performs at least as well, so I will use that method to perform parameter tuning. I do not have any special insight as to the possible range of parameters, so I will take the conservative route and draw the parameters from a uniform distribution.

Because it is more costly to predict a terrorist attack when one doesn't happen than it is to predict that one doesn't occur (but then it actually takes place), I will use recall as my evaluation metric. It would also be interesting to check out the AUROC, but that can be left for another time.

In [88]:
from scipy.stats import randint as sp_randint
from sklearn.metrics import recall_score

tree_params = {'min_samples_split': sp_randint(2,15)}

neighbors_params = {'n_neighbors': sp_randint(1,15)}

# First we'll do some tuning and training on the nearest neighbors model.
# I've kept the number of iterations at the default value, but this can be tweaked as well.
random_search = RandomizedSearchCV(k_neighbors, param_distributions=neighbors_params,scoring='recall',cv=5)
random_search.fit(X,y)


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x108b38128>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='recall', verbose=0)

In [101]:
random_search.best_score_

0.6513832119777242

The best nearest neighbor model I trained identifies about 65 percent of the positive samples as positive, which is better than random chance, but can still use some work. Now let's try the decision tree.

In [104]:
random_search = RandomizedSearchCV(dtree, param_distributions=tree_params,scoring='recall', cv=5)
random_search.fit(X,y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x108a5ae10>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='recall', verbose=0)

In [105]:
random_search.best_score_

1.0

This score is too good to be true. Let's see what happens when we train and tune for AUROC instead.

In [106]:
from sklearn.metrics import roc_auc_score
random_search = RandomizedSearchCV(dtree, param_distributions=tree_params,scoring='roc_auc', cv=5)
random_search.fit(X,y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x108a5ae10>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [107]:
random_search.best_score_

1.0

I still believe that the score here is way too good to be true. In this situation, I would investigate whether or not there is some systematic bias in how the data was collected or if there was some bug in my data cleaning.